# transmart python client with JupyterLab integration
A short demonstration on getting data from tranSMART into the Jupyter Notebook analytical environment.

In [1]:
import transmart as tm
from transmart.api.v2.dashboard import Dashboard

## Getting a user account

Before being able to connect to our demo environment, make sure you have account in our user management system. You are able to self register.

Link to create account: [KeyCloak registration](https://keycloak-dwh-test.thehyve.net/auth/realms/transmart/protocol/openid-connect/auth?response_type=code&client_id=transmart-client)

If you are unable to create an account, you can use the dummy account and password: ``demo-user`` 

# User account details
You have to fill in you user details here. 

In [2]:
username = None
password = None

Generate an offline token for your user:

In [3]:
import requests

kc_token_url = "https://keycloak-dwh-test.thehyve.net/auth/realms/transmart-dev/protocol/openid-connect/token"
kc_client_id= "transmart-client"

r = requests.post(url=kc_token_url,
                  data=dict(grant_type='password',
                            client_id=kc_client_id,
                            scope= 'offline_access',
                            username=username,
                            password=password
                           )
                 )
offline_token = r.json().get('refresh_token')
offline_token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ0Mko0OGRxWU9JY1BMYkQ3Q091UjktSFF6ZFo3TnlHS3hKcHdZeFprNlRFIn0.eyJqdGkiOiJlYzhiZGE1ZS01YTk3LTQwOTEtYTM1OS1hNzdmMDU3ODg5YzIiLCJleHAiOjAsIm5iZiI6MCwiaWF0IjoxNTYyMTQwODM0LCJpc3MiOiJodHRwczovL2tleWNsb2FrLWR3aC10ZXN0LnRoZWh5dmUubmV0L2F1dGgvcmVhbG1zL3RyYW5zbWFydC1kZXYiLCJhdWQiOiJ0cmFuc21hcnQtY2xpZW50Iiwic3ViIjoiZDkyZWEyYjItYmE4OC00MTY5LWIzOTItM2MyYWMyNDFmMWEwIiwidHlwIjoiT2ZmbGluZSIsImF6cCI6InRyYW5zbWFydC1jbGllbnQiLCJhdXRoX3RpbWUiOjAsInNlc3Npb25fc3RhdGUiOiJhZTRmOWRmNi0zODI2LTRmYjQtYmZlNi1kNGUxZGExNmQwMzEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7InJlYWxtLW1hbmFnZW1lbnQiOnsicm9sZXMiOlsiaW1wZXJzb25hdGlvbiIsInZpZXctdXNlcnMiLCJxdWVyeS1ncm91cHMiLCJxdWVyeS11c2VycyJdfSwidHJhbnNtYXJ0LWNsaWVudCI6eyJyb2xlcyI6WyJST0xFX0FETUlOIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19fQ.TD6pE-nekr5s_MgLOa9CJcRYGcYFtJrKe8JWt9iDSgu2MQ6WZvh-

First create a connection with a transmart instance with V2 api enabled. This could take a litlle time as some caches are built.

In [4]:
api = tm.get_api(
    host = 'https://transmart-dev.thehyve.net',
    kc_url = "https://keycloak-dwh-test.thehyve.net",
    kc_realm = "transmart-dev",
    offline_token = offline_token,
    print_urls=True
)

https://transmart-dev.thehyve.net/v2/studies
https://transmart-dev.thehyve.net/v2/tree_nodes?depth=0&counts=False&tags=True
No valid cache found. Building indexes...
Finished in 0.24 seconds
https://transmart-dev.thehyve.net/v2/pedigree/relation_types


The main objects to query transmart are created here. This `ObservationConstraint` object can be used specified and combined to create queries to the api.

In [5]:
c = api.new_constraint(study='ORACLE_1000_PATIENT', concept='O1KP:NUM39')
c

ObservationConstraint(concept='O1KP:NUM39', study='ORACLE_1000_PATIENT')

An example call that gets the counts for the chosen constraints

In [6]:
c.observations.counts()

https://transmart-dev.thehyve.net/v2/observations/counts


{'observationCount': 1200, 'patientCount': 1200}

Further specify the constraint using intuitive attributes.

In [7]:
c.min_value = 15
c

ObservationConstraint(concept='O1KP:NUM39', study='ORACLE_1000_PATIENT', min_value=15)

In [8]:
c.observations.counts()

https://transmart-dev.thehyve.net/v2/observations/counts


{'observationCount': 202, 'patientCount': 202}

In [9]:
print('Possible attributes to specify constraints:\n- ', end='')
print('\n- '.join(sorted(c.params.keys())))

Possible attributes to specify constraints:
- concept
- max_date_value
- max_start_date
- max_value
- min_date_value
- min_start_date
- min_value
- study
- subject_set_id
- trial_visit
- value_list


In [10]:
api.observations.aggregates_per_concept(c)

https://transmart-dev.thehyve.net/v2/observations/aggregates_per_concept


{'aggregatesPerConcept': {'O1KP:NUM39': {'numericalValueAggregates': {'avg': 17.600906671769305,
    'count': 202,
    'max': 27.1052065269,
    'min': 15.0155800246,
    'stdDev': 2.437859947351904}}}}

In [11]:
obs = api.observations(c)

https://transmart-dev.thehyve.net/v2/observations?type=clinical&constraint={"args": [{"type": "concept", "conceptCode": "O1KP:NUM39"}, {"type": "study_name", "studyId": "ORACLE_1000_PATIENT"}, {"type": "value", "valueType": "NUMERIC", "operator": ">=", "value": 15}], "type": "and"}


In [12]:
obs.dataframe.head()

,concept.conceptCode,concept.conceptPath,concept.name,numericValue,patient.age,patient.birthDate,patient.deathDate,patient.id,patient.inTrialId,patient.maritalStatus,patient.race,patient.religion,patient.sex,patient.sexCd,patient.subjectIds.SUBJ_ID,patient.trial,study.name
0,O1KP:NUM39,\Public Studies\Oracle_1000_Patient\Numerical ...,numerical_39,17.852166,65,None,None,-2199,subject_1199,None,None,None,male,Male,O1KP:2199,ORACLE_1000_PATIENT,ORACLE_1000_PATIENT
1,O1KP:NUM39,\Public Studies\Oracle_1000_Patient\Numerical ...,numerical_39,15.961849,65,None,None,-2198,subject_1198,None,None,None,female,Female,O1KP:2198,ORACLE_1000_PATIENT,ORACLE_1000_PATIENT
2,O1KP:NUM39,\Public Studies\Oracle_1000_Patient\Numerical ...,numerical_39,16.056566,65,None,None,-2196,subject_1196,None,None,None,female,Female,O1KP:2196,ORACLE_1000_PATIENT,ORACLE_1000_PATIENT
3,O1KP:NUM39,\Public Studies\Oracle_1000_Patient\Numerical ...,numerical_39,19.280176,65,None,None,-2193,subject_1193,None,None,None,female,Female,O1KP:2193,ORACLE_1000_PATIENT,ORACLE_1000_PATIENT
4,O1KP:NUM39,\Public Studies\Oracle_1000_Patient\Numerical ...,numerical_39,15.829366,65,None,None,-2189,subject_1189,None,None,None,female,Female,O1KP:2189,ORACLE_1000_PATIENT,ORACLE_1000_PATIENT


## Integration with widgets and bqplot

Methods `find_concept()` and `interact()` can be used to visually create and modify a constraint object.

In [13]:
c.find_concept('study:oracle')

In [14]:
c.interact()

And there is dashboard for exploration.

In [15]:
dash = Dashboard(api, patients=c)

https://transmart-dev.thehyve.net/v2/patient_sets?name=ObservationConstraint(concept='O1KP:NUM39', study='ORACLE_1000_PATIENT', min_value=15)
https://transmart-dev.thehyve.net/v2/observations/counts_per_study_and_concept


In [16]:
dash.get()